In [1]:
from langchain_core.messages import SystemMessage, ToolMessage, AIMessage, HumanMessage
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, START, END

from typing_extensions import TypedDict
import os
import uuid
from typing import Dict

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['MISTRAL_API_KEY'] = os.getenv('MISTRAL_API_KEY')

model = "mistral-small-latest"
llm = ChatMistralAI(
    model=model,
    temperature=0,
    max_retries=5
)

In [3]:
sys_prompt = "You are a helpful assistant that solves math problems."
hum_prompt = "Solve 2x+3=7 for x."

# Invoke LLM
response = llm.invoke([
    SystemMessage(content=sys_prompt),
    HumanMessage(content=hum_prompt)]
).content.strip()

print(response)

To solve the equation 2x + 3 = 7 for x, follow these steps:

1. Subtract 3 from both sides of the equation:
   2x + 3 - 3 = 7 - 3
   2x = 4

2. Divide both sides by 2:
   2x / 2 = 4 / 2
   x = 2

So, the solution is x = 2.


In [4]:
# State definition
class State(TypedDict):
    messages: Dict

In [5]:
def read_file(file_path):
    with open(file_path, 'r') as f:
        return f.read()

In [6]:
cv_path = "data/cv.txt"
job_path = "data/job_description.txt"
start_state = {"messages": {"cv_path": cv_path, "job_path": job_path}}